LSTM univariate 1h

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

df = pd.read_csv('dataset.csv',sep = ';', nrows = 17544)

a = [str(date) for date in df['datetime']]
a = [date[:-6] for date in a]
date_time = pd.to_datetime(a)

df.loc[:,['datetime']] = date_time
df = df.set_index('datetime') 
df = df[['precio_spot']]
# Feature scaling

from sklearn.preprocessing import MinMaxScaler
sc = MinMaxScaler(feature_range = (0, 1))
scaled_set = sc.fit_transform(df[['precio_spot']])

ModuleNotFoundError: No module named 'numpy'

In [2]:

X_train = []
y_train = []
X_val = []
y_val = []
X_test = []
y_test = []


train_hours = int(0.7*(len(df))+2)
validation_hours = int(0.15*(len(df)))
test_hours = int(0.15*(len(df)))

train_hours + validation_hours + test_hours == len(df) 

True

In [3]:

n_lags = 48

# Training set
for i in range(n_lags, train_hours): # len(X_train) = 1258 - 60 = 1198
    X_train.append(scaled_set[i-n_lags:i, 0])
    y_train.append(scaled_set[i, 0])
X_train, y_train = np.array(X_train), np.array(y_train)

len(X_train)

# Validation set
for i in range(train_hours, train_hours + validation_hours): # len(X_train) = 1258 - 60 = 1198
    X_val.append(scaled_set[i-n_lags:i, 0])
    y_val.append(scaled_set[i, 0])
X_val, y_val = np.array(X_val), np.array(y_val)

len(X_val)

# Test set
for i in range(train_hours + validation_hours, len(df)): # len(X_train) = 1258 - 60 = 1198
    X_test.append(scaled_set[i-n_lags:i, 0])
    y_test.append(scaled_set[i, 0])
X_test, y_test = np.array(X_test), np.array(y_test)

# X_test = np.reshape(X_test, (X_test.shape[1], X_test.shape[2], 1))
# y_test = np.reshape(y_test, (y_test.shape[1], 1))

len(X_test)

# Reshaping
X_train.shape
y_train.shape

X_val.shape
y_val.shape

X_test.shape
y_test.shape

X_train = np.reshape(X_train, (X_train.shape[0], X_train.shape[1], 1))
y_train = np.reshape(y_train, (y_train.shape[0], 1))

X_val = np.reshape(X_val, (X_val.shape[0], X_val.shape[1], 1))
y_val = np.reshape(y_val, (y_val.shape[0], 1))

X_test = np.reshape(X_test, (X_test.shape[0], X_test.shape[1], 1))
y_test = np.reshape(y_test, (y_test.shape[0], 1))



In [4]:
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import Dropout
from tensorflow.keras.callbacks import ModelCheckpoint

# Initializing the RNN
regressor = Sequential()

# Adding first LSTM layer and some Dropout regularisation
regressor.add(LSTM(units = 50, return_sequences = True, input_shape = (X_train.shape[1], 1)))
regressor.add(Dropout(0.2))

# Adding a second LSTM layer and some Dropout regularisation
regressor.add(LSTM(units = 50, return_sequences = True))
regressor.add(Dropout(0.2))

# Adding a second LSTM layer and some Dropout regularisation
regressor.add(LSTM(units = 50, return_sequences = True))
regressor.add(Dropout(0.2))

# Adding a third LSTM layer and some Dropout regularisation
regressor.add(LSTM(units = 50, return_sequences = False))
regressor.add(Dropout(0.2))

# Adding an output layer
regressor.add(Dense(units=1))

regressor.summary()

ModuleNotFoundError: No module named 'keras'

In [ ]:

# Fitting the RNN to the training set
regressor.fit(X_train, y_train, validation_data=(X_val, y_val), epochs = 20, batch_size = 32, callbacks=[cp1]) # Batch size to be loaded in the RAM indirectly 

plt.plot(range(len(regressor.history.history['loss'])), regressor.history.history['loss'])
plt.xlabel('Epoch number')
plt.ylabel('Loss')
plt.show()

In [ ]:
test_predictions = regressor.predict(X_test)


In [ ]:

# TEST
test_predictions = []
test_predictions = regressor.predict(X_test)
# test_predictions = sc.inverse_transform(test_predictions)
test_results = pd.DataFrame(data={'Test Predictions':test_predictions, 'Actuals':y_test}, index = test_index)
test_results

# Plotting test set
plt.plot(test_results['Test Predictions'])
plt.plot(test_results['Actuals'])

plt.plot(y_test, color = 'blue')
plt.plot(test_predictions, color = 'green')